<a href="https://colab.research.google.com/github/kacperkurowski/CE888/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 5

##Loading Files

In [14]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", header = None)
print(df)

arr = df.values
print(arr.shape)

rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]
(24983, 101)
1835357 (18

Replacing 10% Values with 99

In [22]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])


new_arr, idx = replace(arr, 0.1)

print(arr[idx[0][0], idx[1][0]])
print(new_arr[idx[0][0], idx[1][0]])



-8.5
99.0


LFM of the gidden ratings

In [31]:
n_latent_factors = 5

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

#print(latent_user_preferences)
#print(latent_item_features)

def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=5):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not rating == 99.0:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

sgd()  # Note how the MSE decreases with the number of iterations

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

45.60694798450255


array([[ 7.38734167e+01,  6.66869469e-01,  2.04632959e-01, ...,
         4.14385014e-01, -7.10683130e-02,  1.02132767e+00],
       [ 9.86215248e+01,  1.19001913e+00,  4.78235102e-01, ...,
         1.00195469e+00,  4.17544934e-03,  1.46399290e+00],
       [ 5.84126138e+01,  6.11937021e-01,  1.02230278e-01, ...,
         5.73473239e-01,  6.99611429e-02,  9.51142642e-01],
       ...,
       [ 4.23176355e+01,  4.48644113e-01,  3.87898777e-02, ...,
         3.64609967e-01, -1.94575938e-02,  6.41854489e-01],
       [ 4.08108623e+01,  3.20752584e-01,  1.53199041e-01, ...,
         3.16107873e-01, -8.73191262e-02,  5.21778619e-01],
       [ 7.18026016e+01,  6.30726755e-01,  8.81617534e-02, ...,
         5.77965208e-01, -2.91961610e-02,  1.06684395e+00]])

Comparing the data

In [33]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 73.87341674821668)","(-7.82, 0.6668694688871327)","(8.79, 0.20463295899960773)","(-9.66, 0.3274459555851788)","(-8.16, -1.1656137690455701)","(-7.52, 0.24896457933013658)","(-8.5, 1.3551594495440984)","(-9.85, -0.3744062205549458)","(4.17, -0.4965060739709272)","(-8.98, -0.36843349134061665)","(99.0, 1.199591325023841)","(-8.5, 1.7112220216964267)","(-6.75, 1.4108014664501334)","(-7.18, -1.5228312826225519)","(8.45, 1.3141063306845262)","(99.0, -1.6954042912353444)","(-7.52, -2.9178358707155128)","(-7.43, -1.0666776351307112)","(-9.81, -0.6180575043013438)","(-9.85, 0.1417010090999657)","(-9.85, -1.0602257444801217)","(-9.37, 2.145166080901187)","(1.5, 0.915114831167369)","(-4.37, 0.08787071966966675)","(-9.81, -1.5086968927287108)","(-8.5, 0.33206592051949535)","(1.12, 1.164238980692751)","(7.82, 3.1554617704843655)","(2.86, 1.5098958682768602)","(9.13, 2.841409464536021)","(-7.43, -0.3189065253524991)","(2.14, 2.0056364930905874)","(-4.08, 2.92110370281757)","(99.0, -1.1697623322570918)","(7.82, 0.9951210092252912)","(5.05, 2.942453507159073)","(4.95, 3.221859967962735)","(-9.17, -1.1775271290440452)","(-8.4, 1.261752720401188)","(-8.4, 1.0378333558516295)",...,"(8.59, 2.3159255803632637)","(3.59, 2.844987854185371)","(-6.84, 0.3675243689403405)","(-9.03, -0.3850236703899434)","(2.82, 2.350589691980919)","(-1.36, 2.481316146177951)","(-9.08, -0.6042103346607)","(8.3, 2.4008324987142924)","(5.68, 2.5170799500341077)","(-4.81, 0.6303791518677887)","(99.0, -0.81594612931335)","(99.0, 2.476254730318744)","(99.0, 0.7410547344856206)","(99.0, -1.0300313116061883)","(99.0, -0.08754520278848518)","(99.0, 2.009221147540712)","(99.0, 0.6104046614825064)","(-9.42, 1.6127624522373014)","(99.0, 0.11006565962450782)","(99.0, 0.7616104552225569)","(99.0, 1.4929709411411163)","(-7.72, 0.9026971365215093)","(99.0, 1.802089102447347)","(99.0, 0.37149744908902244)","(99.0, 0.5994130587423508)","(99.0, 0.34365046330290905)","(99.0, 1.3833318194251443)","(99.0, 1.706251578847062)","(99.0, 2.6260501338207134)","(99.0, 0.47740681250644973)","(2.82, 1.6806811506561021)","(99.0, 0.80491750284196)","(99.0, 1.7401905330299239)","(99.0, 1.0633005942363476)","(99.0, 0.9211905316712864)","(99.0, 1.266145343047472)","(-5.63, 1.5640132844449706)","(99.0, 0.41438501378647874)","(99.0, -0.07106831303684016)","(99.0, 1.0213276725906943)"
1,"(100.0, 98.62152483492491)","(4.08, 1.1900191291972082)","(-0.29, 0.47823510190257656)","(99.0, 0.43288707542279664)","(4.37, -1.5903614961782098)","(-2.38, 0.5572663129181944)","(-9.66, 1.86739810916666)","(-0.73, -0.4706168647925308)","(-5.34, -0.7752867399387997)","(8.88, -0.6238623562672817)","(9.22, 1.55419599325412)","(6.75, 2.2406768214599913)","(8.64, 1.8568916835407303)","(4.42, -2.2460737788450453)","(7.43, 1.7311061089464983)","(4.56, -2.23546787945042)","(-0.97, -3.8799218438971916)","(4.66, -1.3578537317225998)","(-0.68, -0.8550234228637562)","(3.3, 0.16244238894513255)","(-1.21, -1.2510233594177778)","(0.87, 2.8709647923348047)","(8.64, 1.2065582557028107)","(8.35, 0.05619966104475895)","(9.17, -1.9401621135649838)","(0.05, 0.5888178041602399)","(7.57, 1.5721929691836778)","(4.71, 4.011818709409957)","(0.87, 1.8681468815665263)","(-0.39, 3.8923451946741627)","(6.99, -0.6218737141964606)","(6.5, 2.91587176667343)","(-0.92, 3.992008993978049)","(7.14, -1.5106900401972052)","(9.03, 1.2052835888504219)","(-1.8, 3.759526770617276)","(0.73, 4.176554885605483)","(7.09, -1.5973902007862124)","(3.4, 1.6102746043759952)","(-0.87, 1.5215677254653572)",...,"(-6.7, 3.008163350429516)","(99.0, 3.69223896109206)","(-9.03, 0.2527126168970524)","(4.47, -0.8203963865297038)","(4.08, 3.00111815982023)","(-3.83, 3.1770234274664952)","(8.74, -0.9655030418701939)","(1.12, 3.329347739126704